In this notebook, we gather two pieces of annotation: one csv is the 2K groundtruth, one is our snorkel gen output. See how the PP precision goes.
    
    

In [33]:
'''
combine the generative onset df_gen
'''

sub_dfs=[]
for train_bucket in list(range(11))+[13,14]+list(range(16,35))+[36]:
    sub_dfs+=[pd.read_csv(open("data/annotations_2k/onset_split"+str(train_bucket)+"_window0.csv","r"))]
df = pd.concat(sub_dfs)
df=df.drop(columns=['Unnamed: 0'])
df.to_csv(open("data/annotations_2k/onset_joint_split.csv","w"))

df2=df.copy()
import ast
import numpy as np

def transform_gm_label(x):
    x=ast.literal_eval(x)
    if abs(x[0]-x[1])<1e-4 and abs(x[1]-x[2])<1e-4:
        return [0,0,1]
    elif np.argmax(x)==0:
        return [1,0,0]
    elif np.argmax(x)==1:
        return [0,1,0]
    return [0,0,1]

df2['winningHighlight'] = df2['winningHighlight'].apply(lambda x: str(transform_gm_label(x)))
df2.to_csv(open("data/annotations_2k/onset_joint_split_hard_as_soft.csv","w"))

In [ ]:
'''
Get an order from label level to world level docID
'''

label_level_groundtruth_path = pathlib.Path.cwd().joinpath('data', 'annotations_label-level_all-to-date-2018-4-25-WithTitle.csv')
df_label_level = pd.read_csv(label_level_groundtruth_path)

actual_paperID=df_label_level["paperID"].values

In [22]:
'''
validTypesMapping
'''

import numpy as np
import re
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support

validTypes = set(['Method', 'Background', 'Findings', 'Mechanism', 'Purpose'])
validTypesMapping=defaultdict(lambda:2)
validTypesMapping["Method"]=2
validTypesMapping["Background"]=2
validTypesMapping["Findings"]=2
validTypesMapping["Mechanism"]=1
validTypesMapping["Purpose"]=0

In [35]:
count_PP_MN=0
count_PP=0
count_MN=0
count_none=0
# print(df2.groupby('paperID').size())
for paperID, paperData in df2.groupby('paperID'):
    paperData = paperData.sort_values(by="globalPsn")
#     print(paperData)
    aspects=paperData['winningHighlight'].tolist()
    
    
    hasPP=False
    hasMN=False
    for aspect in aspects:
        aspect=np.argmax(ast.literal_eval(aspect))
        if aspect==0:
            hasPP=True
        if aspect==1:
            hasMN=True
            
    if hasPP and hasMN:
        count_PP_MN+=1
    elif hasPP:
        count_PP+=1
    elif hasMN:
        count_MN+=1
    else:
        count_none+=1
print(count_PP_MN,count_PP,count_MN,count_none)
    

4892 1459 1466 641


In [24]:
'''
Get the mapping from paperID to docID
'''
paperID_docID_mapping=dict()

for index, row in df_groundtruth.iterrows():
    paperID_docID_mapping[row['paperID']]=row['docID']


In [25]:
print("len(paperID_docID_mapping)",len(paperID_docID_mapping))

len(paperID_docID_mapping) 2071


In [8]:
flags = ["not good","but","weird","no"]

def is_normal(meta_note, flags):
    '''
    heuristics for whether an abstract is "suitable"
    '''
    if not meta_note.lower().startswith("make") and not meta_note.lower().startswith("empirical"):
        return False
    for flag in flags:
        if flag in meta_note.lower():
            return False
    return True


import pandas as pd

df_normal_mapping=pd.read_excel("data/upwork_abstract-suitability_notes.xlsx")
doc_normal_mapping=dict()
for index,row in df_normal_mapping.iterrows():
    if row['docID'] not in doc_normal_mapping or doc_normal_mapping[row['docID']]:
        doc_normal_mapping[row['docID']]=is_normal(row['meta_note'],flags)
print("len(doc_normal_mapping)",len(doc_normal_mapping))


len(doc_normal_mapping) 1646


In [9]:
import pandas as pd
import pathlib
import numpy as np
import csv
import ast
from IPython.core.display import display, HTML

# gen_path = pathlib.Path.cwd().joinpath('data', 'annotations_2k','onset_joint_split_hard_as_soft.csv')
# df_gen = pd.read_csv(gen_path)

def render_text(text,labels,label_mapping=None):
    assert len(text)==len(labels)
    html_string=""
    for idx in range(len(text)):
        html_string+="<font style='background-color: "+str(label_mapping[labels[idx]])+";' color='black'>"+text[idx]+" </font>"
    display(HTML(html_string))

df_gen=df2
y_true=[]
y_pred=[]
idx=0
count=0
label_mapping={0:"#ffcccc",1:"#ccd9ff",2:"transparent"}
    
for paperID, paperData in df_gen.groupby('paperID'):
    
    newID=actual_paperID[int(paperID.split("2K_dev_")[1])]
    docID=paperID_docID_mapping[newID]
    if (docID in doc_normal_mapping and doc_normal_mapping[docID] and count<50):
        print()
    else:
        continue

    paperData = paperData.sort_values(by="globalPsn")

    groundtruth_gb=df_groundtruth.groupby('paperID')
    paperData_groundtruth=groundtruth_gb.get_group(actual_paperID[int(paperID.split("2K_dev_")[1])]).sort_values(by="globalPsn")
    contents=paperData_groundtruth['content'].tolist()
    aspects=paperData_groundtruth['winningHighlight'].tolist()
    sequences=[]

    if len(contents)!=len(aspects):
        print("Error, contents' length not equal aspect tags' length")
        continue

    # this handles cases like "however," where the comma is appended
    for pair in zip(contents,aspects):
        content=re.sub(r"\s+","",str(pair[0]))
        if len(content)==0:
            continue
        aspect=validTypesMapping[pair[1]]

        if re.match("\d",content):
            sequences+=[(content,aspect)]
        elif content=="e.g.,":
            sequences+=[("e.g.",aspect)]
            sequences+=[(",",aspect)]
        else:
            output=list(re.findall(r'(\w+)(\W+)*', content))
            token_list=(list(sum(output,())))
            for token in token_list:
                if len(token.strip())>0:
                    sequences+=[(token,aspect)]
            if (len(token_list)==0) and content!="'":
                sequences+=[(content,aspect)]

    contents_gen=paperData['content'].tolist()
    aspects_gen=paperData['winningHighlight'].tolist()
    sequences_gen=[]
    for pair in zip(contents_gen,aspects_gen):
        content=re.sub(r"\s+","",str(pair[0]))
        if len(content)==0:
            continue
        gen_label=ast.literal_eval(pair[1])
        aspect=np.argmax(gen_label)
        if abs(gen_label[0]-gen_label[1])<1e-3 and abs(gen_label[1]-gen_label[2])<1e-3:
            aspect=2
        if content=="'":
            continue
        sequences_gen+=[(content,aspect)]
    if len(sequences)!=len(sequences_gen):
        continue
    
    count+=1
    text=[pair[0] for pair in sequences]
    predicted=[pair[1] for pair in sequences_gen]
    groundtruth=[pair[1] for pair in sequences]
    print(newID)
    display(HTML("<h5>Prediction</h5>"))
    render_text(text,predicted,label_mapping)
    display(HTML("<h5>Groundtruth</h5>"))
    render_text(text,groundtruth,label_mapping)
    display(HTML("==============================<br/>"))



2017_10



msa_2406209768





msa_2407481741




msa_1538715442



msa_2411751434


...


msa_2547405038



msa_2547936469




msa_2548908563



msa_2549624392



msa_2550075934


In [34]:
gen_path = pathlib.Path.cwd().joinpath('data', 'annotations_2k','onset_joint_split_hard_as_soft.csv')
df_gen = pd.read_csv(gen_path)
df_list=[]
count=0
for paperID, paperData in df_gen.groupby('paperID'):
#     newID=actual_paperID[int(paperID.split("2K_dev_")[1])]
#     docID=paperID_docID_mapping[newID]
#     print(newID)

    paperData = paperData.sort_values(by="globalPsn")

    contents_gen=paperData['content'].tolist()
    aspects_gen=paperData['winningHighlight'].tolist()
    sequences_gen=[]
    current_aspect=2
    hasMN=False
    hasPP=False
    for pair in zip(contents_gen,aspects_gen):
        content=re.sub(r"\s+","",str(pair[0]))
        if len(content)==0:
            continue
        gen_label=ast.literal_eval(pair[1])
        aspect=np.argmax(gen_label)
            
        if content=="'":
            continue
            
        if aspect!=2 and current_aspect!=aspect:
            if aspect==1:
                hasMN=True
            if aspect==0:
                hasPP=True
            current_aspect=aspect
        sequences_gen+=[(content,current_aspect)]
        if content==".":
            current_aspect=2
        
    if not hasMN or not hasPP:
        continue
    else:
        count+=1
    
    for idx,pair in enumerate(sequences_gen):
        
        winningHighlight=[0,0,0]
        winningHighlight[pair[1]]=1
        
        df_list+=[[winningHighlight,paperID,idx+1,pair[0]]]

print(count)

df=pd.DataFrame(df_list,columns=['winningHighlight', 'paperID', 'globalPsn','content'])
df.to_csv(open(str(gen_path).replace(".csv","_expanded.csv"),"w"))   # in the spreadsheet, punctuations are splitted

4892
